In [26]:
!wget 'https://raw.githubusercontent.com/krishnaik06/SpamClassifier/master/smsspamcollection/SMSSpamCollection'

--2021-05-26 06:14:47--  https://raw.githubusercontent.com/krishnaik06/SpamClassifier/master/smsspamcollection/SMSSpamCollection
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 477907 (467K) [text/plain]
Saving to: ‘SMSSpamCollection.1’

SMSSpamCollection.1 100%[===================>] 466.71K  --.-KB/s    in 0.09s   

2021-05-26 06:14:47 (5.20 MB/s) - ‘SMSSpamCollection.1’ saved [477907/477907]



In [27]:
import pandas as pd
data = pd.read_csv('/content/SMSSpamCollection',sep='\t',names=['labels','message'])
data

,labels,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [28]:
import nltk
import re

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
porter = PorterStemmer()
corpus = []
for text in data['message']:
  d = re.sub(r'[^a-zA-Z]',' ',text)
  d = d.lower()
  d = d.split()
  d = [porter.stem(word) for word in d if word not in set(stopwords.words('english'))]
  d = ' '.join(d)
  corpus.append(d)
print(corpus[:5])

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat', 'ok lar joke wif u oni', 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli', 'u dun say earli hor u c alreadi say', 'nah think goe usf live around though']


In [30]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus)
X = pd.DataFrame(X.A,columns=cv.get_feature_names())
print(X.head())

   aah  aathi  abi  abil  abiola  abj  ...  yunni  yuo  yup  zed  zindgi  zoe
0    0      0    0     0       0    0  ...      0    0    0    0       0    0
1    0      0    0     0       0    0  ...      0    0    0    0       0    0
2    0      0    0     0       0    0  ...      0    0    0    0       0    0
3    0      0    0     0       0    0  ...      0    0    0    0       0    0
4    0      0    0     0       0    0  ...      0    0    0    0       0    0

[5 rows x 2500 columns]


In [33]:
y = pd.get_dummies(data['labels'])
print(y)

      ham  spam
0       1     0
1       1     0
2       0     1
3       1     0
4       1     0
...   ...   ...
5567    0     1
5568    1     0
5569    1     0
5570    1     0
5571    1     0

[5572 rows x 2 columns]


In [34]:
y = y.iloc[:,1].values
y

array([0, 0, 1, ..., 0, 0, 0], dtype=uint8)

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,stratify=y,random_state=1)

In [36]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

In [38]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

0.9849246231155779
[[1194   12]
 [   9  178]]
